<a href="https://colab.research.google.com/github/Manu1ND/colab/blob/main/best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
pip uninstall opencv-python-headless==4.5.5.62
pip install opencv-python-headless==4.5.2.52
pip install gdown
apt-get install zip unzip
'''

# Specs

In [1]:
  !nvidia-smi

Sat Feb 26 08:11:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.4 GB  | Proc size: 141.0 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [3]:
import warnings
warnings.filterwarnings('ignore')

# Import dependencies

In [4]:
''' !pip install --upgrade gupload
# from pydrive.auth import GoogleAuth
from google.colab import auth
auth.authenticate_user() '''

# Run below two lines in terminal
''' !pip uninstall opencv-python-headless==4.5.5.62
!pip install opencv-python-headless==4.5.2.52 '''
!pip install fastai -q --upgrade
!pip install --upgrade albumentations
from fastai.vision.all import *
import gdown
import os
''' 
from google.colab import output
output.clear() '''

In [5]:
print(torch.__version__)

1.10.0+cu111


# Download Dataset

In [7]:
gdown.download('https://drive.google.com/u/1/uc?id=1PsX8ULvWg-peKtUduyTqkyxfxXoom13S', './owl.zip', quiet=False)
!unzip ./owl.zip
!rm ./owl.zip

''' output.clear() '''

# Preprocessing

In [7]:
seed = 1000

def seed_everything(seed = seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

In [8]:
path = Path('./Dataset_OWL_COD10K_v3/')
train_images = path/'Train/Image'
train_labels = path/'Train/GT'
test_name = "Test"
test_images = path/test_name/'Image'
test_labels = path/test_name/'GT'

In [9]:
def get_y_fn(x): 
  return Path(str(x).replace("Image","GT").replace(".jpg",".png"))

In [10]:
def ParentSplitter(x):
    return Path(x).parent.name==test_name

In [11]:
codes = ['background','owl']
name2id = {v:k for k,v in enumerate(codes)}
print(name2id)
void_code = name2id['owl']

{'background': 0, 'owl': 1}


In [12]:
def foreground_acc(input, target):
    target = cast(target.squeeze(1), TensorBase)
    mask = target != void_code
    return (input.argmax(dim=1)[mask]==target[mask]).float().mean()

In [13]:
from albumentations import (
    Compose,
    
    # Pixel
    HueSaturationValue,
    Posterize,
    RandomSunFlare,

    # Spatial
    GridDistortion,
    HorizontalFlip,
    Perspective,
    SafeRotate
)

In [14]:
class SegmentationAlbumentationsTransform(ItemTransform):
    split_idx = 0
    def __init__(self, aug): 
        self.aug = aug
    def encodes(self, x):
        img,mask = x
        aug = self.aug(image=np.array(img), mask=np.array(mask))
        return PILImage.create(aug["image"]), PILMask.create(aug["mask"])

In [15]:
class TargetMaskConvertTransform(ItemTransform):
    def __init__(self): 
        pass
    def encodes(self, x):
        img,mask = x
        mask = np.array(mask)
        mask[mask==255]=1
        mask = PILMask.create(mask)
        return img, mask

# Parameters

In [16]:
bs = 8
splitter = RandomSplitter(valid_pct=0.2, seed=seed)
item_tfms = [Resize((512, 512), method=ResizeMethod.Crop), TargetMaskConvertTransform()]
batch_tfms = [HueSaturationValue(), Posterize(), RandomSunFlare(), GridDistortion(), HorizontalFlip(), Perspective(), SafeRotate()]
arch = resnet50
pretrained = True
metrics = [foreground_acc, Dice(), JaccardCoeff()]
act_cls = Mish
self_attention = True
lr = 1e-3
wd = 1e-1
epochs = 200

In [17]:
monitor_training = "valid_loss"
comp_training = np.less

monitor_evaluating = "dice"
comp_evaluating = np.greater

patience = 50

In [18]:
transformPipeline = Compose(batch_tfms, p=1)
transformPipeline = SegmentationAlbumentationsTransform(transformPipeline)
item_tfms.append(transformPipeline)

In [19]:
callbacksFitBeforeUnfreeze = [
    ShowGraphCallback(),
    EarlyStoppingCallback(monitor = monitor_training, comp = comp_training, patience = patience),
    SaveModelCallback(monitor = monitor_training, comp = comp_training, every_epoch = False, fname = "Camonet_Best")  
]

# Data Block

In [ ]:
dblock = DataBlock(blocks = (ImageBlock, MaskBlock(codes)),
                  get_items = get_image_files,
                  splitter = splitter,
                  get_y = get_y_fn,
                  item_tfms = item_tfms,
                  batch_tfms = [Normalize.from_stats(*imagenet_stats)]
                  )
  
dls = dblock.dataloaders(train_images, bs=bs, num_workers=0)

In [ ]:
dls.show_batch(cmap='binary_r', vmin=0, vmax=1)

# Training

In [ ]:
seed_everything()

In [ ]:
learn = unet_learner(dls, arch, pretrained = pretrained, metrics = metrics, wd=wd,
                    act_cls = act_cls, self_attention = self_attention)


In [ ]:
learn.summary()

In [ ]:
learn.fit_one_cycle(epochs, lr, cbs=callbacksFitBeforeUnfreeze)

In [ ]:
learn.show_results(cmap='binary_r', vmin=0, vmax=1)

In [ ]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~Training Result~~~~~~~~~~~~~~~~~~~~~~~~~")
learn.validate()

# Testing 

In [ ]:
test_img = get_image_files(test_images)
dl = learn.dls.test_dl(test_img, with_labels=True)

In [ ]:
dl.show_batch(max_n=9, cmap='binary_r', vmin=0, vmax=1)

In [ ]:
_,_,preds = learn.get_preds(dl=dl, with_decoded=true)

In [ ]:
learn.show_results(dl=dl, shuffle=False, max_n=9, cmap='binary_r', vmin=0, vmax=1)

In [ ]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~Testing Result~~~~~~~~~~~~~~~~~~~~~~~~~~")
learn.validate(dl=dl)

# Export .pkl

In [ ]:
learn.export()

# Upload .pth and .pkl to drive

In [ ]:
!gupload --to '1IKUHdIiDgHPxeJt5gKhGmzrnxUJIAVG1' "/content/models/Camonet_Best.pth"

In [ ]:
!gupload --to '1IKUHdIiDgHPxeJt5gKhGmzrnxUJIAVG1' "/content/export.pkl"